<a href="https://colab.research.google.com/github/t-nakatani/signate_stu22/blob/exp/signate_stu_v41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

v33でfocal-lossのgamma=3.0  
CV=, LB=

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive
drive.mount('/content/drive')
ROOT = "/content/drive/MyDrive/Colab Notebooks/signate/"

#学習用データと評価用データの読み込み
train = pd.read_csv(os.path.join(ROOT, "train.csv")).drop_duplicates(subset='description')
test = pd.read_csv(os.path.join(ROOT, "test.csv"))

jobdic = dict(zip([1, 2, 3, 4], ['DataScientist', 'ML Engineer', 'Software Engineer', 'Consultant']))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Fri Aug 19 07:12:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import re
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_tag(text)
        # #アルファベット以外をスペースに置き換え
        clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        #単語長が1文字以下のものは削除する
        clean_short_tokenized = [word for word in clean_punc.split() if len(word) > 1]
        #単語同士をスペースでつなぎ, 文章に戻す
        clean_text = ' '.join(clean_short_tokenized)
        
        clean_texts.append(clean_text)
    return clean_texts

def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)

train['description'] = cleaning(train['description'])
test['description'] = cleaning(test['description'])

In [ ]:
!pip install -q transformers==3

     |████████████████████████████████| 754 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 31.1 MB/s 
     |████████████████████████████████| 1.3 MB 60.7 MB/s 
     |████████████████████████████████| 880 kB 69.9 MB/s 


In [ ]:
import numpy as np
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import RobertaTokenizer, RobertaModel
# from transformers import BertModel, AutoTokenizer 

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
# from transformers import BertTokenizer, BertModel
from torch import optim
from torch import cuda
import time
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score

In [ ]:
# seeds
SEED = 2022
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(SEED)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    current_device = torch.cuda.current_device()
    print("Device:", torch.cuda.get_device_name(current_device))

Device: Tesla P100-PCIE-16GB


In [ ]:
# Datasetの定義
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):  # len(Dataset)で返す値を指定
    return len(self.X)

  def __getitem__(self, index):  # Dataset[index]で返す値を指定
    text = self.X[index]
    inputs = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      pad_to_max_length=True,
      truncation=True
    )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    labels = self.y[index]

    return {
      'ids': torch.LongTensor(ids),
      'mask': torch.LongTensor(mask),
      'labels': torch.Tensor(labels)
    }

1. from_pretrainedの文字列でconfig.from_pretrainedを呼び出し
2. config.from_pretrainedの結果をconfigにセット

In [ ]:
# BERT分類モデルの定義
class BERTClass(torch.nn.Module):
  def __init__(self, drop_rate, otuput_size):
    super().__init__()
    self.config = AutoConfig.from_pretrained("roberta-base", num_labels=4)
    self.bert = AutoModelForSequenceClassification.from_pretrained("roberta-base", config=self.config)

  def forward(self, ids, mask):
    out = self.bert(ids, attention_mask=mask)
    return out[0]

In [ ]:
from torchvision.ops import sigmoid_focal_loss

class multi_focal_loss(torch.nn.Module):
    def __init__(self, gamma):
      super(multi_focal_loss, self).__init__()
      self.gamma = gamma

    def forward(self, outputs, targets): 
      focal_loss = sigmoid_focal_loss(outputs, targets, gamma=self.gamma)
      return focal_loss.mean()

In [ ]:
def calculate_loss_and_f1_score(model, criterion, loader, device, epoch, max_epoch):
  """ 損失・正解率を計算"""
  model.eval()
  loss = 0.0
  total = 0
  correct = 0
  all_labels = []
  all_preds = []

  with torch.no_grad():
    with tqdm(loader) as pbar:
      pbar.set_description('valid')
      for i, data in enumerate(pbar):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)


        outputs = model(ids, mask)
        loss += criterion(outputs, labels).item()

        preds = torch.argmax(outputs, dim=-1).cpu().numpy() # バッチサイズの長さの予測ラベル配列
        labels = torch.argmax(labels, dim=-1).cpu().numpy()  # バッチサイズの長さの正解ラベル配列
        all_labels += labels.tolist()
        all_preds += preds.tolist()
        total += len(labels)
        correct += (preds == labels).sum().item()
        pbar.set_postfix(loss=loss/(i+1), f1=f1_score(all_labels, all_preds, average="macro"))

  return loss / len(loader), f1_score(all_labels, all_preds, average="macro")

In [ ]:
def train_val(train_loader, val_loader, model_path):
  device = 'cuda' if cuda.is_available() else 'cpu'
  model = BERTClass(DROP_RATE, OUTPUT_SIZE).to(device)
  criterion = multi_focal_loss(gamma=GAMMA)
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
  
  log_valid = []
  best_acc = 0
  all_labels = []
  all_preds = []

  # train
  for epoch in range(NUM_EPOCHS):
    print('\nepoch:', epoch+1)
    loss_train = correct = total = 0
    model.train()
    with tqdm(train_loader) as pbar:
      pbar.set_description('train')
      for i, data in enumerate(pbar):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        labels = data['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(ids, mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()
        preds = torch.argmax(outputs, dim=-1).cpu().numpy()
        labels = torch.argmax(labels, dim=-1).cpu().numpy()

        all_labels += labels.tolist()
        all_preds += preds.tolist()

        total += len(labels)
        correct += (preds == labels).sum().item()
        pbar.set_postfix(loss=loss_train/(i+1), f1=f1_score(all_labels, all_preds, average="macro"))

    loss_valid, f1_valid = calculate_loss_and_f1_score(model, criterion, val_loader, device, epoch, NUM_EPOCHS)
    log_valid.append([loss_valid, f1_valid])
    if best_acc < f1_valid:
      best_acc = f1_valid
      
      torch.save(model, os.path.join(ROOT, model_path))
      print('==== model saved ====')

  return log_valid

In [ ]:
DROP_RATE = 0.1
OUTPUT_SIZE = 4
BATCH_SIZE = 16
NUM_EPOCHS = 20
LEARNING_RATE = 2e-6
N_CV = 5
GAMMA = 3.0
VERSION = 'v41'

In [ ]:
import sklearn.metrics
def train_val_predict(
        df_train,       # 学習用のデータ
        text_column,    # 対象のカラム名
        target_column,  # 目的変数のカラム名
        df_valid=None,  # 検証用データ
        df_val_test=None,       # 予測用データ
        model_file_prefix="",  # 保存時のファイル名識別子
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
    ):
  
    # model
    device = 'cuda' if cuda.is_available() else 'cpu'
    model_path = "{}_{}.pth".format(model_file_prefix, VERSION)
    model = BERTClass(DROP_RATE, OUTPUT_SIZE)
    criterion = multi_focal_loss(gamma=GAMMA)
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
    

    # dataset
    df_val_test[target_column].fillna(0)

    max_len = 512
    tokenizer = AutoTokenizer.from_pretrained("roberta-base")
    dataset_train = CreateDataset(df_train[text_column].values, pd.get_dummies(df_train[target_column]).values, tokenizer, max_len)
    dataset_val = CreateDataset(df_valid[text_column].values, pd.get_dummies(df_valid[target_column]).values, tokenizer, max_len)
    dataset_test = CreateDataset(df_val_test[text_column].values, pd.get_dummies(df_val_test[target_column]).values, tokenizer, max_len)

    dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
    dataloader_valid = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)
    dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

    # train
    print(f'\n================  start train ({model_file_prefix}) ================')
    metric = train_val(dataloader_train, dataloader_valid, model_path)
    print(f'================  end train ({model_file_prefix}) ================\n')

    # predict
    model = torch.load(os.path.join(ROOT, model_path))
    model.eval()
    pred_y_list = []
    emb_list = []
    with torch.no_grad():
      with tqdm(dataloader_test) as pbar:
        pbar.set_description('test')
        for data in pbar:
          ids = data['ids'].to(device)
          mask = data['mask'].to(device)
          labels = data['labels'].to(device)

          output = model.forward(ids, mask)
          pred = torch.argmax(output, dim=-1).cpu().numpy()

          pred_y_list.extend(pred)
          emb_list.extend(output.cpu().numpy())
    return metric, pred_y_list, emb_list

In [15]:
import sklearn.model_selection

text_column, target_column, n_splits = ('description', 'jobflag', N_CV)

df = pd.concat([train, test], ignore_index=True, sort=False)
df_train = df[df[target_column].notnull()]
df_test = df[df[target_column].isnull()]

df_train_idx = df_train.index

# store result
df_pred = pd.DataFrame(df.index, columns=["index"]).set_index("index")
df_emb = pd.DataFrame(df.index, columns=["index"]).set_index("index")
df_emb_pred = None
metric_list = []
all_emb = []

# cross validation
kf = sklearn.model_selection.StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
for i, (train_idx, test_idx) in enumerate(kf.split(df_train, df_train[target_column])):
  df_train_sub = df_train.iloc[train_idx]
  df_test_sub = df_train.iloc[test_idx]

  df_val_test = pd.concat([df_test_sub, df_test], ignore_index=True, sort=False)

  model_file_prefix = "cv_{}".format(i)

  # train
  metric, pred_y_list, emb_list = train_val_predict(
      df_train=df_train_sub, 
      text_column=text_column,
      target_column=target_column, 
      df_valid=df_test_sub,
      df_val_test=df_val_test,
      model_file_prefix=model_file_prefix,
  )
  metric_list.append(metric)
  all_emb.append(emb_list)
  # break

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

valid: 100%|██████████| 19/19 [00:05<00:00,  3.51it/s, f1=0.649, loss=0.0265]



epoch: 20


valid: 100%|██████████| 19/19 [00:05<00:00,  3.52it/s, f1=0.695, loss=0.0279]


==== model saved ====
================  end train (cv_4) ================



test: 100%|██████████| 114/114 [00:32<00:00,  3.51it/s]


In [16]:
all_emb_ = np.array([np.array(emb)[-len(df_test):] for emb in all_emb])

In [17]:
mean_df = pd.DataFrame([[np.mean(arr_val) for arr_val in arr_vec] for arr_vec in np.moveaxis(all_emb_, 0, 2)])
preds = [np.argmax(arr)+1 for arr in mean_df.to_numpy()]
df_submit = pd.DataFrame([df_test.id.to_list(), preds]).T
df_submit.to_csv(os.path.join(ROOT, f'{VERSION}_ensemble.csv'), index=None, header=None)

In [18]:
f1_list = []
for item in metric_list:
  f1_list.append((np.max([acc for loss, acc in item])))
print(np.mean(f1_list))

0.7008481841690205


In [19]:
len(df_submit[df_submit[1] == 2])

58